In [1]:
# Load symbolicregression model

import torch
import os, sys
import symbolicregression
import sympytorch
import requests
from sympy.core.rules import Transform

model_path = "ckpt/model.pt" 
try:
    if not os.path.isfile(model_path): 
        print("Downloading model...")
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        model = torch.load(model_path)
        model = model.cuda()
    print(model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)    
    
est = symbolicregression.model.SymbolicTransformerRegressor(
                        model=model,
                        max_input_points=10001,
                        n_trees_to_refine=5,
                        rescale=True
                        )

cuda:0
Model successfully loaded!


In [2]:
# from transformers import AutoTokenizer
# CONTEXT_LENGTH = 256
# tokenizer = AutoTokenizer.from_pretrained("xhyi/PT_GPTNEO350_ATG") 
# tokenizer.pad_token = tokenizer.eos_token
# from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
# model = AutoModelForCausalLM.from_pretrained("datasets/normalize_symbolic_regression_results_20231215/gptneo-350m-5500.model")


from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained("datasets/normalize_symbolic_regression_results_flant5_20231219/flant5-base-36000-loss0.097.model")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot
from IPython.display import display

# Given an expr f (of variable t), returns its integral, together with t's and y's for regression
def integrate_expr(f, min_x=-4.0, max_x=4.0, increment=0.002, verbose=False):
    if verbose:
        print("Running integration on")
        display(f)
    # Compute integration
    x, t = symbols(['x','t'])
    fi = integrate(f, t)
    if verbose:
        display(fi)
        #plot(fi, (t, min_x, max_x))
    # Generate data for symbolic regression
    fl = lambdify((t), fi, "numpy")
    ts = np.arange(min_x, max_x, increment)
    ys = fl(ts)
    return fi, ts, ys
    
integrate_expr(sympify("2*t**2-t*2+2.5"), verbose=True)

Running integration on


2*t**2 - 2*t + 2.5

0.666666666666667*t**3 - 1.0*t**2 + 2.5*t

(0.666666666666667*t**3 - 1.0*t**2 + 2.5*t,
 array([-4.   , -3.998, -3.996, ...,  3.994,  3.996,  3.998]),
 array([-68.66666667, -68.58170266, -68.49681062, ...,  36.50791852,
         36.56077862,  36.61369466]))

In [4]:
from utils.utils import *

def round_expr(expr, num_digits=2):
    return expr.xreplace(Transform(lambda x: x.round(num_digits), lambda x: isinstance(x, sp.Float)))

# Run symbolic regression on given data
# Returns: (raw regressed expr, rounded expr, model refined expr)
@timeout(15)
def symbolic_regress(sr_model, xs, ys, generate_refinement=True, verbose=False):
    if verbose:
        print("Running Symbolic Regression...")
    ##Example of data
    xs = np.reshape(xs, (len(xs),1))
    ys = np.reshape(ys, (len(ys),1))
    est.fit(xs,ys)
    #
    replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
    model_str = sr_model.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
    for op,replace_op in replace_ops.items():
        model_str = model_str.replace(op,replace_op)
    #
    raw_expr = sp.parse_expr(model_str)
    x_0, t = symbols(['x_0', 't'])
    raw_expr = raw_expr.subs(x_0, t)
    if verbose:
        display(raw_expr)
    #
    expr = sp.expand(raw_expr)
    rounded_expr = round_expr(expr)
    if verbose:
        display(rounded_expr)
    #
    # Encode some input text
    if generate_refinement:
        prompt = str(rounded_expr)
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        #
        # Generate text
        output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.1)
        #
        # Decode and print the output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_expr = sympify(generated_text)
        if verbose:
            display(generated_expr)
        #
    else:
        generated_expr = None
    return raw_expr, rounded_expr, generated_expr


fi, ts, ys = integrate_expr(sympify("2*t**2-t*2+2.5"), verbose=False)
rounded_fi = round_expr(fi)
display(rounded_fi)
raw_expr, rounded_expr, generated_expr = symbolic_regress(est, ts, ys, verbose=False)
display(generated_expr)

print("Diff1:")
display(rounded_fi-rounded_expr)
print("Diff2:")
display(rounded_fi-generated_expr)

0.67*t**3 - 1.0*t**2 + 2.5*t

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


0.67*t**3 - t**2 + 2.5*t

Diff1:


0

Diff2:


-7.812500000004e-5*t**3

In [5]:
import json
import random
import numpy as np

def load_expressions(filepaths):
    lines = []
    for filepath in filepaths:
        fin = open(filepath, 'r')
        lines.extend(fin.readlines())
        fin.close()
    #
    random.shuffle(lines)
    exprs = set()
    for line in lines:
        data = json.loads(line)
        for k,v in data.items():
            if k in ('f_t', 'g_t'):
                try:
                    if 'sqrt' not in v[1]:
                        expr = sympify(v[1])
                        exprs.add(expr)
                except:
                    continue
    return exprs

#exprs = load_expressions(['datasets/parametric_equations.json'])  #'datasets/function_evaluation.json', 

In [6]:
# seen_exprs = set()
# fin = open("datasets/parametric_equations_integral_results.json", "r")
# lines = fin.readlines()
# for line in lines:
#     result = json.loads(line)
#     expr = result["original"]
#     seen_exprs.add(expr)
# fin.close()
# print(f"{len(seen_exprs)} exprs loaded")

# fout = open("datasets/parametric_equations_integral_results.json", "a")
# num_seen = 0
# num_seen_changed = False

# for f in exprs:
#     if str(f) in seen_exprs:
#         num_seen+=1
#         num_seen_changed = True
#         continue
#     else:
#         num_seen_changed = False
#     if num_seen_changed:
#         print(f"{num_seen} exprs ignored")
#     #print("Original expr and its integral:")
#     #display(f)
#     #print(f)
#     try:
#         fi, xs, ys = integrate_expr(f, verbose=False)
#         x, t = symbols(['x','t'])
#         fi = fi.subs({x:t})
#         rounded_fi = round_expr(fi)
#         #display(rounded_fi)
#         raw_expr, rounded_expr, generated_expr = symbolic_regress(est, xs, ys, generate_refinement=True, verbose=False)
# #         print("Generated expr:")
# #         display(generated_expr)
#         results = {"original":str(f),
#                    "integral":str(fi),
#                    "rounded_integral":str(rounded_fi),
#                    "regressed":str(raw_expr),
#                    "rounded_regressed":str(rounded_expr),
#                    "generated_regressed":str(generated_expr),
#                    "diff_rounded": str(rounded_fi-rounded_expr),
#                    "diff_generated": str(rounded_fi-generated_expr)
#                   }
#         fout.write(json.dumps(results))
#         fout.write('\n')
#         fout.flush()
#     except:
#         print("Failed to run symbolic regression")
#         continue
#     #     print("Diff1:")
#     #     display(rounded_fi-rounded_expr)
#     #     print("Diff2:")
#     #     display(rounded_fi-generated_expr)

# fout.close()

In [ ]:
from sympy import evalf, N
from utils.utils import *


# Check if f1 and f2 are almost equal.
# Note: Relative error is defined based on f1. Please use the original expression as f1.
def almost_equal(f1, f2, max_abs_error=0.011, max_relative_error=0.011, verbose=False):
    expr = f1-f2
    coeff_pairs = None
    try:
        coeff_pairs = get_coefficients_and_exponents(expr)
    except:
        print("Cannot get_coefficients_and_exponents")
        print(str(expr))
    if coeff_pairs is None:
        constants = get_all_constants(expr)
    else:
        constants = [p[0] for p in coeff_pairs]
    # Check if all diffs are within max_abs_error
    violators = [c for c in constants if c == sp.nan or abs(c) > max_abs_error]
    if verbose:
        print("Violating constants:", violators)
    if len(violators) == 0:
        return True
    # Check if all violating diffs are within max_relative_error
    try:
        coeffs1 = get_polynomial_coeffs(f1)
        coeffs2 = get_polynomial_coeffs(f2)
    except:
        return False
    for i in range(len(coeffs1)):
        if abs(coeffs1[i] - coeffs2[i]) > max_abs_error and \
           abs(coeffs1[i] - coeffs2[i]) > max_relative_error*abs(coeffs1[i]):
            return False
    return True
    

fin = open("datasets/parametric_equations_integral_results.json", "r")
lines = fin.readlines()

num_total, qualified_rounded, qualified_generated = 0, 0, 0
for line in lines:
    result = json.loads(line)
    if "diff_rounded" not in result or "diff_generated" not in result:
        continue
    rounded_integral = N(sympify(result["rounded_integral"]))
    rounded_regressed = N(sympify(result["rounded_regressed"]))
    try:
        rounded_regressed = filter_non_polynomial(rounded_regressed)
    except:
        print("Cannot filter non-polynomials on", str(rounded_regressed))
    #generated_regressed = N(sympify(result["generated_regressed"]))
    diff_rounded = rounded_integral - rounded_regressed
    #diff_generated = sympify(result["diff_generated"])
    num_total += 1
    if almost_equal(rounded_integral, rounded_regressed, verbose=True):
        qualified_rounded += 1
    else:
        display(rounded_integral)
        display(rounded_regressed)
        print(rounded_regressed)
        display(diff_rounded)
#     if is_close_to_zero(diff_generated, True):
#         qualified_generated += 1
    
fin.close()

print(num_total, qualified_rounded, qualified_generated)

Violating constants: [-0.10999999999999943]
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: [0.03, -0.019999999999999574, -0.710000000000008]


0.11*t**3 + 5.33*t**2 + 85.33*t

0.08*t**3 + 5.35*t**2 + 86.04*t

0.08*t**3 + 5.35*t**2 + 86.04*t


0.03*t**3 - 0.0199999999999996*t**2 - 0.710000000000008*t

...

0.04*t**3 - 2.56*t**2 + 58.78*t

-2.55*t**2 + 59.07*t

-2.55*t**2 + 59.07*t


0.04*t**3 - 0.0100000000000002*t**2 - 0.289999999999999*t

Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating constants: []
Violating consta

In [ ]:
from sympy import evalf, N
f = sympify("-1.17*t**3 - 2*t**2/3 + 7.66*t + 0.15") #2*t/(0.29*t + 44.02)")

variables = list(f.free_symbols)
t = variables[0]

#(x,x.is_constant())
#print([[x for x in term.as_coeff_exponent(t)] for term in f.as_ordered_terms()])
    

g = filter_non_polynomial(f)
get_polynomial_coeffs(g)



In [ ]:
f=sympify("0.55*t**3 + 1.46*t**2*arctan(75.4) - 21.56*t**2 + 0.96*t*arctan(75.4)**2 - 28.5*t*arctan(75.4) + 267.05*t")
sympify("sin(75.4)")